# What's the Price of Wheat?

<img src="http://www.ictinternational.com/content/uploads/2015/02/wheat1.jpg" width=640 height=480/>

## Project Background and Motivation

We chose to create a predictor of the stock price of wheat because of the ever-pervasive fluctuation in food prices. Wheat is among one of the most fundamental agricultural commodities in the United States, so understanding and ultimately predicting the price of this commodity will allow us to understand an essential part of our economic ecosystem. Our team has diverse backgrounds in engineering and science, so we wanted to choose a topic that has profound global implications. One of the grand challenges identified by the leaders of the U.N. and world bank is the shortage of food in our ever growing population. We thought it would be interesting for us to be able to build predictive forecasting of the stock price of this key commodity as this framework is a proof of principle for forecasting the price of any other agricultural commodity of choice.

In [1]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

## 1. Getting Our Data

### 1.1 Importing Data

We start by importing datasets of our various features, such as rainfall, temperature, and fertilizer use. Most of these datasets already have some raw datasets available to download, which we have included in our project folder. We will hold off on adding soil moisture, since that requires some involved data scraping.

In [23]:
feedgrainsdf=pd.read_csv("FeedGrains.csv")
nitdf=pd.read_csv("nitrogen_fertilizer.csv")
phosdf=pd.read_csv("phosphate_fertilizer.csv")
potdf=pd.read_csv("potash_fertilizer.csv")
w1df=pd.read_csv("wheatyearbooktable01full.csv")
w2df=pd.read_csv("wheatyearbooktable04full.csv")
dirtyprecdf=pd.read_csv("stationprec.csv")

In [24]:
dirtytempdf=pd.read_csv("stationtemp.csv")

### 1.2 Cleaning DataFrames

Now that we have a bunch of dataframes, next up is to clean it, hopefully into something that we can use. For example, dirtyprecdf is currently unusable. 

#### 1.2.1 Cleaning Precipitation Data

## NOTE: Do not run unless masochistic. The cleaned dataframe is down below. This (most likely unoptimized) cleaning will take a long time.

There are a few things in the precipitation database that we don't want. For one, station IDs aren't really helpful to us, and instead we want to convert this to the states that each station is in. In addition, the numbers in each month need to be interpreted, since it is often in the form ####F. There are also some results with -9999M, which we want to get rid of, since those indicate missing data. We want to get rid of the letters and turn the number into an understandable inch unit. According to the readme from our source, each number is the number of 1/100ths of an inch (e.g. 1486 = 14.86 inches)

In [7]:
dirtyprecdf.head(10)

,Station ID,Year,Jan,Feb,March,April,May,June,July,August,September,October,November,December
0,AQC00914000,1981,4279,3745,10762,6067,4096,3606,6203,5292,3092,6866,7163,7866
1,AQC00914000,1982,5039,9643,3211,2016,3355,2827,3199,9356,4150,6418,3965,1595
2,AQC00914000,1983,3351,2971,3044,2642,1644,1717,1020,1788,3433,6801,2531,7242
3,AQC00914000,1984,3368,3538,8187,2715,2916,3288,1246,3391,2932,6578,4787,9787
4,AQC00914000,1985,5202,3078,3279,8414,2884,4787,3447,3193,5296,5410,3950,1651
5,AQC00914000,1986,8573,4212,2799F,7095F,5012F,3135F,3878F,2989F,6368F,4740F,4312F,8310F
6,AQC00914000,1987,5146,7648,3954F,3573F,3118F,2788F,2043F,3498F,841,2669,2298,6208F
7,AQC00914000,1988,3336F,4798,4908,4400F,4297F,2817,3878,2548F,3792,4645,5969F,8915F
8,AQC00914000,1989,5523F,5806F,3273F,5388F,3672F,3079F,4166F,209,788,4982,6379,3731
9,AQC00914000,1990,4932,6857,3288,4518,1635,3598,2214,1541,2611,5439,4525,3530


In [92]:
precdf=dirtyprecdf
# remove the -9999M rows, the F, and turn numbers to inch values
for col in precdf:
    if (col!="Station ID" and col!= "Year"):
        precdf= precdf[precdf[col] != '-9999M']
        precdf[col]=precdf[col].map(lambda x: x.rstrip("F"))
        precdf[col]=precdf[col].map(lambda x: float(x)/100.00)
precdf.head(10)

C:\Users\Vinh\Anaconda\lib\site-packages\IPython\kernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Vinh\Anaconda\lib\site-packages\IPython\kernel\__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Station ID,Year,Jan,Feb,March,April,May,June,July,August,September,October,November,December
0,AQC00914000,1981,42.79,37.45,107.62,60.67,40.96,36.06,62.03,52.92,30.92,68.66,71.63,78.66
1,AQC00914000,1982,50.39,96.43,32.11,20.16,33.55,28.27,31.99,93.56,41.50,64.18,39.65,15.95
2,AQC00914000,1983,33.51,29.71,30.44,26.42,16.44,17.17,10.20,17.88,34.33,68.01,25.31,72.42
3,AQC00914000,1984,33.68,35.38,81.87,27.15,29.16,32.88,12.46,33.91,29.32,65.78,47.87,97.87
4,AQC00914000,1985,52.02,30.78,32.79,84.14,28.84,47.87,34.47,31.93,52.96,54.10,39.50,16.51
5,AQC00914000,1986,85.73,42.12,27.99,70.95,50.12,31.35,38.78,29.89,63.68,47.40,43.12,83.10
6,AQC00914000,1987,51.46,76.48,39.54,35.73,31.18,27.88,20.43,34.98,8.41,26.69,22.98,62.08
7,AQC00914000,1988,33.36,47.98,49.08,44.00,42.97,28.17,38.78,25.48,37.92,46.45,59.69,89.15
8,AQC00914000,1989,55.23,58.06,32.73,53.88,36.72,30.79,41.66,2.09,7.88,49.82,63.79,37.31
9,AQC00914000,1990,49.32,68.57,32.88,45.18,16.35,35.98,22.14,15.41,26.11,54.39,45.25,35.30


In [7]:
# turn station ID into state they are found in
statiddf=pd.read_csv("statid.csv")
statiddf=statiddf[['Station ID', 'State']]
statid={}
for index, row in statiddf.iterrows():
    statid[row['Station ID']]=row['State']
# hard code some values that aren't in the station id list, but appear in precdf
statid['USC00085612']='FL'

In [93]:
%%time
testdf=pd.concat([precdf.head(10),precdf.tail(10)])
precdf.rename(columns={'Station ID':'State'}, inplace=True)
for index, row in precdf.iterrows():
    if index%1000==0:
        print index
    precdf['State'][index]=statid[row['State']]

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
22000
23000
24000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000
159000
16000

C:\Users\Vinh\Anaconda\lib\site-packages\IPython\kernel\__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [97]:
precdf.to_csv("precdf.csv", index=False)

## Continue BELOW to continue cleaning data

In [3]:
precdf=pd.read_csv("precdf.csv")

In [4]:
prec_grouped = precdf.groupby(['State','Year']).mean().reset_index()
prec_grouped

,State,Year,Jan,Feb,March,April,May,June,July,August,September,October,November,December
0,AK,1981,15.683362,8.257069,8.708879,3.853276,4.915259,6.550172,10.535862,15.486552,12.651207,12.820948,12.226121,7.486121
1,AK,1982,5.100342,3.618462,5.281624,5.847863,6.187179,5.819829,7.964359,6.752137,15.152222,11.733419,9.297863,9.896068
2,AK,1983,7.984655,6.574655,2.432241,7.119828,6.262931,4.087155,6.538879,13.819310,10.171466,13.382414,8.525172,3.358017
3,AK,1984,12.432069,8.805259,6.767759,5.168362,4.187759,6.101121,8.322500,10.346638,8.663448,9.372586,7.268966,10.093190
4,AK,1985,15.284348,7.119652,8.681217,5.548087,5.936609,7.537130,5.375565,10.198435,13.984087,9.077913,4.656261,15.266783
5,AK,1986,12.364872,6.162650,6.593077,4.190427,4.491368,4.831368,7.465470,11.689744,8.029231,18.506838,10.268120,15.018718
6,AK,1987,13.020000,7.321282,5.140513,5.975641,6.503419,10.063504,6.694017,5.087863,17.302222,17.963675,13.330855,10.894957
7,AK,1988,7.215470,9.894017,9.811880,7.323504,6.320000,6.074530,6.445641,11.865556,10.154017,14.196752,10.109829,13.272650
8,AK,1989,8.603621,2.369569,2.484310,4.185862,7.217845,6.950517,6.059138,10.700517,14.211724,15.089397,10.749397,15.118621
9,AK,1990,9.548205,7.374872,7.928034,3.385556,3.875385,5.460855,7.563504,10.194103,18.809231,10.304786,6.216667,11.274872


#### 1.2.1 Cleaning Precipitation Data

Now that we have cleaned the data for the precipitation, we follow a similar procedure for temperature. Since the temperature data is from the same source, we will follow almost exactly the same procedure.

### Again, this takes a LONG time, 1-2 hours, so skip down to next heading if you want

In [39]:
dirtytempdf=pd.read_csv("stationtemp.csv")

In [40]:
# remove the -9999M rows, convert to degrees celsius
for col in dirtytempdf:
    if (col!="Station" and col!= "Year"):
        dirtytempdf= dirtytempdf[dirtytempdf[col] != '-9999M']
        dirtytempdf[col]=dirtytempdf[col].map(lambda x: float(x)/100.00)

In [ ]:
# find averages for each state each year, and replace 'Station' with 'State'
# trying to optimize runtime, so storing last station ID, and State value to minimze list searching
%%time
last_stat=('statid','state')
dirtytempdf.rename(columns={'Station':'State'}, inplace=True)
i=0
for index, row in dirtytempdf.iterrows():
    if index%10000==0:
        print index
    if (last_stat[0] != dirtytempdf['State'][index]):
        state=statid[row['State']]
        last_stat=(dirtytempdf['State'][index],state)
        dirtytempdf['State'][index]=state
    else:
        dirtytempdf['State'][index]=last_stat[1]
dirtytempdf.head()

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000

In [ ]:
tdf=dirtytempdf.groupby(['State','Year']).mean().reset_index()

In [21]:
dirtytempdf

,State,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,AS,1981,27.50,27.00,26.90,26.70,26.85,26.05,26.00,26.10,26.85,26.45,27.00,27.10
1,AS,1982,27.65,26.75,27.85,27.50,27.00,26.75,25.85,26.00,26.10,26.95,26.65,27.15
2,AS,1983,27.75,28.50,27.55,27.20,27.15,26.35,25.80,25.60,26.65,26.30,26.80,27.35
3,AS,1984,27.00,27.40,27.25,27.45,27.50,26.85,26.00,26.40,26.30,26.30,27.10,26.70
4,AS,1985,26.75,27.45,27.80,27.00,26.85,26.60,26.30,26.50,26.40,27.00,26.85,27.50
5,AS,1986,27.20,27.45,27.75,27.20,26.90,26.95,26.15,25.60,26.50,27.20,27.55,27.50
6,AS,1987,27.50,28.00,27.80,27.70,26.80,25.75,25.60,25.55,26.40,26.75,27.55,27.80
7,AS,1988,28.00,28.10,28.15,27.35,27.50,27.35,26.30,26.80,26.60,26.60,26.40,26.50
8,AS,1989,26.40,26.80,27.30,27.00,26.85,26.65,25.80,26.40,27.50,27.15,26.85,26.75
9,AS,1990,27.40,27.35,27.45,26.85,27.60,26.80,26.65,26.65,26.75,27.35,27.50,27.55
